In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [2]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

# Note: Use a random state of 1 for each algorithm to ensure consistency between tests
rand_stat = 1

In [6]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df_base = pd.read_csv(file_path, skiprows=1)[:-2]

C:\Users\me\anaconda3\envs\MachLearn\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
# take all rows, only columns in the list defined above; make a copy; put it back into df
df = df_base.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status  -- presumably because these haven't had a chance to fail yet
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# wouldn't grace period be the same?
grace_mask = df['loan_status'] != 'In Grace Period'
df = df.loc[grace_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100

In [10]:
# converting variety of string labels into binary 0=bad, 1=good for machine learning

bin_labels = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default'], 0)    
bin_labels['Current'] = 1

df = df.replace(bin_labels)

df.reset_index(inplace=True, drop=True)

# Rescale, Split the Data into Training and Testing

In [14]:
# Create our features
X = pd.get_dummies( df.drop('loan_status', axis=1) )

# Create our target
y = df[target]

In [15]:
# rescale data, using min/max for easier compute and because of many 0/1 columns
from sklearn.preprocessing import MinMaxScaler
data_scaler = MinMaxScaler()

# some examples say, fit training data, then transform training and test data; i wonder why.
X_scaled = data_scaler.fit_transform(X)

In [16]:
# split into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=rand_stat)